In [1]:
import vk
import numpy as np
import sklearn 
import statistics as stat
import math
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from collections import Counter
import gc
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
gc.collect()

0

In [2]:
session = vk.Session()
api = vk.API(session)
api.users.get(user_ids=1)
my_id = '55423942'
f_interest = 'sex, bdate, city, universities, schools, interests'

#'verified, sex, bdate, city,country, home_town, online, lists, domain, has_mobile, contacts, 
#site, education, universities, schools, status, last_seen, followers_count, occupation, nickname, relatives, relation, personal, connections, exports, wall_comments, activities, interests, music, movies, tv, books, games, about, quotes, can_post, can_see_all_posts, can_see_audio, can_write_private_message, can_send_friend_request, is_favorite, is_hidden_from_feed, timezone, screen_name, maiden_name,  is_friend, friend_status, career, military, blacklisted, blacklisted_by_me'

## Предсказание даты рождения

#### Извлечение года рождения, если он есть в информации о пользователе

In [3]:
def extract_data(data):
    if 'deactivated' in data:
        return 0
    if 'bdate' in data:
        res_year = data['bdate']
        res_year = res_year.split('.')
        if len(res_year) >= 3:
            res_year = int(res_year[2])           
            if res_year > 1940: return res_year
    return 0        

#### Извлечение числа и месяца рождения.

Обычно их пользователь указывает сам, в отличии от года. Также была идея определять по поздравлениям на стене, но обычно, если у пользователя открыта стена, то на ней указаны дд:мм

In [4]:
def extract_data_1_2(data):
    if 'deactivated' in data:
        return 0
    if 'bdate' in data:
        res_year = data['bdate']
        res_year = res_year.split('.')
        res_d = res_year[0]
        res_m = res_year[1]
        return (res_d + '.' + res_m)
    return 0        

####  Предсказание года рождения при помощи медианы и моды годов рождения друзей

In [5]:
def predict_bdate (unknow_id):
    count = 0
    feature = []
    res = api.users.get(user_id = unknow_id, fields = f_interest)    
    if extract_data_1_2(res[0]): print("bdate:", extract_data_1_2(res[0]))
    res_year = extract_data(res[0])
    if res_year: 
        print ('year:', res_year)
        return 
   
    friends = api.friends.get(user_id = unknow_id, order = 'hist', fields = f_interest)
    bdata = []
    year = []
    for i in range(0, len(friends)):
        y = extract_data(friends[i])
        if y: year.append(y)      
    year_f = min(stat.median(year), Counter(year).most_common(1)[0][0])
    year_t = max(stat.median(year), Counter(year).most_common(1)[0][0])
    
    print ("year from", year_f - 1, "to",year_t + 1)
    return (year_f + year_t)/2

In [6]:
predict_bdate('86975991')

bdate: 1.10
year from 1992.0 to 1995


1993.5

#### Предсказание года рождения на основе таких признаков, как:
среднее, медиана, мода, минимальный возраст и максимальный возраст среди друзей, количество друзей

In [263]:
from sklearn.ensemble import RandomForestRegressor
rgr = RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [267]:
def predict_bdate_1 (unknow_id):
    count = 0
    feature = []
    res_year = -1
    res = api.users.get(user_id = unknow_id, fields = f_interest)
    if extract_data_1_2(res[0]): print("bdate:", extract_data_1_2(res[0]))
    us_year = extract_data(res[0])
    if us_year:
        print ("year:", us_year)
        return
    data = []
    friends = api.friends.get(user_id = unknow_id, order = 'hist', fields = f_interest)
    for i in range(0, len(friends)):
        if extract_data(friends[i]):
            year = []
            fr_friends = api.friends.get(user_id = friends[i]['user_id'], order = 'hist', fields = f_interest)
            for j in range(0, len(fr_friends)):
                y = extract_data(fr_friends[j]) 
                if y:
                    year.append(y)
            if(len(year) > 0):
                data.append(extract_data(friends[i]))
                data.append(stat.mean(year))
                data.append(stat.median(year))
                data.append(Counter(year).most_common(1)[0][0])
                data.append(min(year))
                data.append(max(year))
                data.append(len(fr_friends))
                #api.account.getAppPermissions(access_token = my_access_token, user_id = unknow_id)
                #groups = api.groups.get(access_token = my_access_token, user_id = unknow_id, extended = 1)
                #data.append(len(groups))
        #if i % 10 == 0: print(i)
    data = np.array(data).reshape((-1, 7))
    #print(data)
    X_train = data[:, 1:]
    y_train = data[:, 0]
    X_test = []
    X_test.append(stat.mean(y_train))
    X_test.append(stat.median(y_train))
    X_test.append(Counter(y_train).most_common(1)[0][0])
    X_test.append(min(y_train))
    X_test.append(max(y_train))
    X_test.append(len(friends))
    X_test = np.array(X_test).reshape((-1, 6))
    rgr.fit(X_train, y_train)
    year = rgr.predict(X_test)
    
    print ("approximate year:", year[0])

In [268]:
predict_bdate_1('86975991')


bdate: 1.10
approximate year: 1990.931


## Предсказание города, в котором проживает пользователь

Будем возвращать самый часто встречаемый город среди друзей пользователя. 

Код будет выдавать город, из которого больше всего знакомств у пользователя. Город в виде номера

In [302]:
def predict_city (unknow_id):
    city = []
    res = api.users.get(user_id = unknow_id, fields = f_interest)
    if 'deactivated' in res[0]:
        return 0
    friends = api.friends.get(user_id = unknow_id, order = 'hist', fields = f_interest)
    if 'city' in res[0]:
        res_city = res[0]['city']
        return res_city
    for i in range(0, len(friends)):
        city.append(friends[i].get('city', 100500))
    city = [x for x in city if x != 100500 and x!= 0]
    res = Counter(city).most_common(2)
    #res.insert(0, (res_city, 0))
    return res

In [299]:
predict_city(my_id)

1

## Предсказание пола пользователя

### Способ_1

Предсказание по соотношению полов в каждом из сообществ, на которые подписан пользователь

In [270]:
def predict_sex (unknow_id):
    res = api.users.get(user_id = unknow_id, fields = f_interest)
    if 'deactivated' in res[0]:
        return 0
    if 'sex' in res[0]:
        s = res[0]['sex']
        if s == 1: return 'woman'
        else: return 'man'
    id_groups = api.users.getSubscriptions(user_id = unknow_id )['groups']['items']
    res = []
    err = 0
    w = 0
    m = 0
    for i in id_groups:
        w0 = 0
        m0 = 0
        people = api.groups.getMembers(group_id = i, fields = f_interest)['users']
        for j in range(0, len(people)):
            if 'deactivated' in people[j]: err += 1
            else:
                if 'sex' in people[j]:
                    s = people[j]['sex']
                    if(s == 1): w0 += 1
                    else: m0 += 1
        w += w0 / (m0 + w0)
        m += m0/(m0 + w0)
        #print(w, m)
    if(w > m): return 'woman'
    else: return 'man'

### Способ_2

Предсказание по поиску имени в Вк и выводу наиболее встречаемого пола. Работает быстрее и не хуже.

In [271]:
def predict_sex_find (unknow_id):
    w = 0
    m = 0
    err = 0
    res = api.users.get(user_id = unknow_id, fields = f_interest)
    if 'deactivated' in res[0]:
        return 0
    first_name = res[0]['first_name']
    people_find = api.users.search(access_token = my_access_token,
                                   q = first_name, sort = 0, count = 100, fields = 'sex, city')
    
    l = len(people_find) - 1
    for i  in range(1, len(people_find)):
        if not 'deactivated' in people_find[i] and 'sex' in people_find[i]:
            if people_find[i]['sex'] == 1: w += 1
            else: m += 1
    if w > m: return 'woman'
    else: return 'man'

## Предсказание интересов.

Будем извлекать интересы друзей и слова из сообществ, на которые подписан пользователь

In [119]:
def predict_interests(unknow_id):
    friends = api.friends.get(user_id = unknow_id, order = 'hist', fields = 'interests')
    s0 = []
    stop_words = stopwords.words('russian')
    stemmer = SnowballStemmer("russian")
    res = api.users.get(user_id = unknow_id, fields = f_interest)
    s_user = []
    if 'interests' in res[0]:
        s = res[0]['interests']        
        s = friends[i]['interests']
        s = s.lower()
        s = re.split('\W+', s)
        if len(s) > 0: s_user += s
    tokens = [i for i in s_user if i not in stop_words and i and i not in string.digits]
    tokens_user = [stemmer.stem(i) for i in tokens]
    
    for i in range(0, len(friends)):
        if 'interests' in friends[i]:
            s = friends[i]['interests']
            s = s.lower()
            s = re.split('\W+', s)
            if len(s) > 0: s0 += s
            #print(s)
            #res_year = res_year.split('.')
    
    
    tokens = [i for i in s0 if i not in stop_words and i and i not in string.digits]
    tokens_fr = [stemmer.stem(i) for i in tokens]
    
    api.account.getAppPermissions(access_token = my_access_token, user_id = unknow_id)
    groups = api.groups.get(access_token = my_access_token, user_id = unknow_id, extended = 1)
    name = []
    for i in range(1, len(groups)):
        s = groups[i]['name'].lower()
        s = re.split('\W+', s)
        name += s
    tokens = [i for i in name if i not in stop_words and i and i not in string.digits]
    tokens_gr = [stemmer.stem(i) for i in tokens]
    
    tokens = tokens_gr + tokens_fr    
    print(Counter(tokens).most_common(15), tokens_user)
    

In [273]:
predict_interests('38506495')

[('игр', 13), ('dot', 12), ('футбол', 11), ('математик', 8), ('музык', 8), ('футбольн', 7), ('москв', 6), ('цска', 6), ('факт', 6), ('подслуша', 5), ('спорт', 5), ('обзор', 5), ('русск', 4), ('физик', 4), ('физтех', 4)] []


## Предсказание вуза. 

Будем рассматривать друзей, у которых возраст +- 4 года от  возраста нашего пользователя

In [288]:
def predict_bdate_w_pr (unknow_id):
    count = 0
    feature = []
    res = api.users.get(user_id = unknow_id, fields = f_interest)    
    res_year = extract_data(res[0])
    if res_year: 
        return res_year
   
    friends = api.friends.get(user_id = unknow_id, order = 'hist', fields = f_interest)
    bdata = []
    year = []
    for i in range(0, len(friends)):
        y = extract_data(friends[i])
        if y: year.append(y)      
    year_f = min(stat.median(year), Counter(year).most_common(1)[0][0])
    year_t = max(stat.median(year), Counter(year).most_common(1)[0][0])

    return (year_f + year_t)/2

In [289]:
def predict_universiry(unknow_id):
    res = api.users.get(user_id = unknow_id, fields = f_interest)
    if 'deactivated' in res[0]:
        return 0
    res_bdate = predict_bdate_w_pr(unknow_id)
    #print(res_bdate, 'data')
    friends = api.friends.get(user_id = unknow_id, order = 'hist', fields = f_interest)
    data_univ = []
    #print(len(friends))
    if 'universities' in res[0] and 'id' in res[0]['universities'][0]:
        res_univ = res[0]['universities'][0]['id']    
    for i in range(0, len(friends)):
        if 'universities' in friends[i]:
            if(len(friends[i]['universities']) >= 1):
                #print(predict_bdate(friends[i]['user_id']))
                if(np.abs(predict_bdate_w_pr(friends[i]['user_id'])  - res_bdate) <= 4):
                    data_univ.append(friends[i]['universities'][0]['name'])
    #print(data_univ)
    return(Counter(data_univ).most_common(5))
    
            
    

In [290]:
predict_universiry('38506495')

[('МФТИ (ГУ)', 16),
 ('НИУ ВШЭ (ГУ-ВШЭ)', 8),
 ('МГУ', 4),
 ('ХНУ им. В. Н. Каразина (бывш. ХГУ им. Горького)', 2),
 ('הטכניון (Technion – IIT)', 1)]